In [1]:
# import ragas.metrics as m
import os
from dotenv import load_dotenv
import pandas as pd
from src.extractor import relationExtractor
from src.llms import OpenAIModel, HuggingFaceLLM
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

load_dotenv()
link = os.getenv('dsa_2214')
token = os.getenv('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN') 
os.environ['OPENAI_API_KEY'] = token

chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [2]:
# data = pd.read_csv('data/sorting.csv')
# data.columns = ['concept', 'outcome']
data = pd.read_csv('data/dsa_clifford_a_shaffer_3_2_java.csv')
data

,chapter,concepts,outcomes
0,Data Structures and Algorithms,Data Structures::Algorithms::Algorithm Efficie...,Demonstrate an understanding of what algorthms...
1,Mathematical Preliminaries,Sets and Relations::Set Notation::Boolean logi...,Demonstrate an understanding of sets::Demonstr...
2,Algorithm Analysis,Asymptotic Analysis::Big O notation::Algorithm...,Demonstrate an understanding of algorithm anal...
3,Lists Stacks and Queues,Lists::Stacks::Queues::Array-based List Implem...,Demonstrate an understanding of Lists::Demonst...
4,Binary Trees,Nodes::Binary Search Tree::Binary Tree::Subtre...,Demonstrate an understanding of binary trees::...
5,Non-Binary Trees,General Tree Traversals::General Tree Node Abs...,Demonstrate an understanding of trees::Demonst...
6,Internal Sorting,Mergesort::Quicksort::Shellsort::Heapsort::Sor...,Demonstrate an understanding of sorting algori...
7,File Processing and External Sorting,Primary Memory::Secondary Storage::Random Acce...,Demonstrate an understanding of disk drives::D...
8,Searching,Searching::Search Queries::Linear Search::Jump...,Demonstrate an understanding of search algorit...
9,Indexing,Indexing::Primary Keys::Secondary Keys::Tree I...,Demonstrate an understanding of primary and se...


In [10]:
actual_concepts = data['concepts'][3].split('::')
actual_concepts

['Lists',
 'Stacks',
 'Queues',
 'Array-based List Implementation Linked Lists',
 'Freelists',
 'Doubly Linked Lists',
 'Dictionary',
 'Circular Queues',
 'Linked Stacks',
 'Linked Queues']

In [11]:
extractor = relationExtractor(link,
                            [chapters[3]], 
                            300, 
                            100, 
                            OpenAIModel())

Created a chunk of size 740, which is longer than the specified 300
Created a chunk of size 395, which is longer than the specified 300
Created a chunk of size 1082, which is longer than the specified 300
Created a chunk of size 393, which is longer than the specified 300
Created a chunk of size 596, which is longer than the specified 300
Created a chunk of size 749, which is longer than the specified 300
Created a chunk of size 1067, which is longer than the specified 300
Created a chunk of size 709, which is longer than the specified 300
Created a chunk of size 502, which is longer than the specified 300
Created a chunk of size 757, which is longer than the specified 300
Created a chunk of size 379, which is longer than the specified 300
Created a chunk of size 665, which is longer than the specified 300
Created a chunk of size 1922, which is longer than the specified 300
Created a chunk of size 389, which is longer than the specified 300
Created a chunk of size 586, which is longer 

In [12]:
from src.metrics import SemanticSimilarity, Contextual_F1
from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric, FaithfulnessMetric, ContextualRelevancyMetric


metrics = [AnswerRelevancyMetric(model = extractor.llm), FaithfulnessMetric(model = extractor.llm), ContextualPrecisionMetric(model = extractor.llm), ContextualRecallMetric(model = extractor.llm), SemanticSimilarity(st_model = extractor.embedding_model)]

In [13]:
# for i in range(5):
result = extractor.evaluate('concepts', 10, actual_concepts, metrics)
print('-' * 20)
print(f'OPENAI MODEL: {extractor.llm.get_model_name()}')
print(f'TEMPERATURE: {extractor.llm.temperature}')
print(f'SENTENCE TRANSFORMER: {extractor.embedding_model}')
print(f'TEXTBOOK: dsa_2214')
print(f'CHAPTERS TESTED: {extractor.chapters}')
print('-' * 20)
for i in result:
    print(f"METRIC: {i['name']} ----> SCORE: {i['score']}")
    print()
    print(f"REASON: {i['reason']}")
    print() 
    print(f"QUERY: {i['input']}")
    print()
    print(f"OUTPUT: {i['output']}")
    print('-' * 20)

/home/zanehutchens/uncc/research/rag/src/extractor.py:240: DeprecationWarning: invalid escape sequence '\s'
  relation = re.sub(re.compile('^[a-zA-Z\s\.,!?]'), '', relation)


AttributeError: you must have at least one of the following functions ran to perform evaluation: identify_concepts, identify_outcomes, identify_key_terms

In [ ]:
full_extractor = relationExtractor(link, 
                                chapters, 
                                500, 
                                100, 
                                OpenAIModel())

In [ ]:
full_extractor.identify_concepts(5)
full_extractor.identify_key_terms(5)
full_extractor.identify_outcomes(5)

In [ ]:
full_extractor.identify_dependencies(full_extractor.key_terms) 

In [ ]:
full_extractor.build_knowledge_graph()

In [ ]:
full_extractor.draw_knowledge_graph()

In [ ]:
full_extractor.build_terminology()

In [ ]:
full_extractor.draw_hierarchy()

In [ ]:
full_extractor.terminology

In [ ]:
from src.retrieval import RetrievalSystem
from langchain_openai import ChatOpenAI

In [ ]:
rs = RetrievalSystem(link, 700, 50, 'msmarco-distilbert-base-tas-b')

In [ ]:
results = rs.pipeline('What is the limits of computation chapter about?', ChatOpenAI(), 4, 10)
results

In [ ]:
ChatOpenAI(model = 'gpt-4o-mini', temperature = 0).invoke(f"Identify 5 key terms from chapter Limits to Computation. The textbook is here: {link}")

In [ ]:
rs.pipeline('Turing Machine, Decidability, Halting Problem, Intractability', ChatOpenAI())

In [ ]:
response = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0).invoke('Provide 5 questions for the topics of Limits of Computation. These should be relatively simple questions. Provide your response as a list of questions.')
response.content

In [ ]:
rs.pipeline('Lists, Stacks, and Queues', ChatOpenAI(model = 'gpt-4o'))

## 